In [1]:
import pandas as pd
df= pd.read_csv('movie_reviews.csv')
df.head(10)

,Summary,Sentiment
0,rock destined st century new conan going make ...,1
1,gorgeously elaborate continuation lord ring tr...,1
2,effective tepid biopic,1
3,sometimes like go movie fun wasabi good place ...,1
4,emerges something rare issue movie honest keen...,1
5,film provides great insight neurotic mindset c...,1
6,offer rare combination entertainment education,1
7,perhaps picture ever made literally showed roa...,1
8,steer turn snappy screenplay curl edge clever ...,1
9,take care cat offer refreshingly different sli...,1


In [2]:
summary = df.Summary.values
labels = df.Sentiment.values 

In [3]:
print(summary)

['rock destined st century new conan going make splash even greater arnold schwarzenegger jean claud van damme steven segal'
 'gorgeously elaborate continuation lord ring trilogy huge column word cannot adequately describe co writer director peter jackson expanded vision j r r tolkien middle earth'
 'effective tepid biopic' ...
 'stand crocodile hunter hurried badly cobbled look godzilla combined scene japanese monster flick canned shot raymond burr commenting monster path destruction'
 'thing look like made home video quickie' 'enigma well made dry placid']


In [4]:
print(labels)

[1 1 1 ... 0 0 0]


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(summary)
vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(summary)
padded_sequence = pad_sequences(encoded_docs, maxlen=200)

In [8]:
#print(tokenizer.word_index)

{'film': 1, 'movie': 2, 'one': 3, 'like': 4, 'story': 5, 'make': 6, 'character': 7, 'time': 8, 'good': 9, 'comedy': 10, 'much': 11, 'even': 12, 'way': 13, 'well': 14, 'director': 15, 'work': 16, 'performance': 17, 'life': 18, 'little': 19, 'funny': 20, 'feel': 21, 'get': 22, 'enough': 23, 'never': 24, 'u': 25, 'bad': 26, 'may': 27, 'love': 28, 'best': 29, 'year': 30, 'thing': 31, 'would': 32, 'look': 33, 'made': 34, 'take': 35, 'come': 36, 'new': 37, 'drama': 38, 'action': 39, 'something': 40, 'could': 41, 'audience': 42, 'really': 43, 'see': 44, 'two': 45, 'minute': 46, 'plot': 47, 'go': 48, 'long': 49, 'still': 50, 'many': 51, 'old': 52, 'every': 53, 'people': 54, 'big': 55, 'nothing': 56, 'first': 57, 'better': 58, 'give': 59, 'end': 60, 'fun': 61, 'without': 62, 'great': 63, 'picture': 64, 'actor': 65, 'though': 66, 'might': 67, 'ever': 68, 'cast': 69, 'world': 70, 'also': 71, 'le': 72, 'real': 73, 'kind': 74, 'another': 75, 'script': 76, 'self': 77, 'humor': 78, 'thriller': 79, 's

In [6]:
print(summary[0])
print(encoded_docs[0])

rock destined st century new conan going make splash even greater arnold schwarzenegger jean claud van damme steven segal
[420, 2427, 2196, 648, 37, 153, 6, 3121, 12, 2722, 1342, 2197, 2198, 1553, 824]


In [7]:
print(padded_sequence[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0  420 2427 2196  648   37  153    6 3121   12 2722 1342
 2197 

In [8]:
# Build the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,     
                                     input_length=200) )
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 32)           522240    
                                                                 
 spatial_dropout1d (SpatialD  (None, 200, 32)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 50)                16600     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 538,891
Trainable params: 538,891
Non-trainable params: 0
__________________________________________________

In [9]:
history = model.fit(padded_sequence,labels,validation_split=0.2, epochs=5, batch_size=32)

Epoch 1/5
267/267 [==============================] - 84s 280ms/step - loss: 0.6410 - accuracy: 0.6370 - val_loss: 0.8209 - val_accuracy: 0.4238
Epoch 2/5
267/267 [==============================] - 71s 267ms/step - loss: 0.4726 - accuracy: 0.7834 - val_loss: 0.7422 - val_accuracy: 0.6174
Epoch 3/5
267/267 [==============================] - 62s 234ms/step - loss: 0.3683 - accuracy: 0.8348 - val_loss: 0.7426 - val_accuracy: 0.6699
Epoch 4/5
267/267 [==============================] - 65s 242ms/step - loss: 0.3194 - accuracy: 0.8629 - val_loss: 0.7736 - val_accuracy: 0.6554
Epoch 5/5
267/267 [==============================] - 42s 156ms/step - loss: 0.2875 - accuracy: 0.8815 - val_loss: 0.8331 - val_accuracy: 0.6582


In [18]:
test_word ="This is soo sad"
tw = tokenizer.texts_to_sequences([test_word])
tw = pad_sequences(tw,maxlen=200)
prediction = int(model.predict(tw).round().item())
labels[prediction]

1